In [ ]:
from sentence_transformers import SentenceTransformer

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
model = SentenceTransformer("infgrad/stella_en_400M_v5", trust_remote_code=True).cuda()

docs = [
    "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
    "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
    "What are some ways to reduce stress?",
    "What are the benefits of drinking green tea?"
]

doc_embeddings = model.encode(docs, device="cuda")


In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

In [ ]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="LLM_embedding",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)

In [ ]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="LLM_embedding",
    wait=True,
    points=[
        PointStruct(id=0, vector=doc_embeddings[0], payload={"text": docs[0]}),
        PointStruct(id=1, vector=doc_embeddings[1].tolist(), payload={"text": docs[1]}),
        PointStruct(id=2, vector=doc_embeddings[2].tolist(), payload={"text": docs[2]}),
        PointStruct(id=3, vector=doc_embeddings[3].tolist(), payload={"text": docs[3]}),
    ],
)

print(operation_info)